In [1]:
import csv
from numpy import *
import itertools

#load the data set
inpdat = "../../../3ormoreDATLOW.csv"
with open(inpdat,'r') as f:
    it = csv.reader(f)
    listit = list(it)

newdata = [listit[0]]
for x in range(1,len(listit)):
    if(float(listit[x][len(listit[0])-1]) == 1.0):
        newdata.append(listit[x])
#filter so dataset is only cimphigh samples
listit = newdata


#threshold support/confidence, change this value to filter rules!
#support: fraction of transactions which contain this item-set
minsupport = .1250
minconfidence = .16


#make candidate set - counts for each item
numsamps = len(listit)-1
c1 = {}
for y in range(1,len(listit[0])-1):
    count = 0
    for x in range(1,len(listit)):
        if(float(listit[x][y]) == 1.0):
            count += 1
    c1[listit[0][y]] = count

c1

{'ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----': 12,
 'AIM2_GRCh37_1:159032487-159032496_Frame-Shift-Del-DEL-TTTTTTTTTT----': 6,
 'ATR_GRCh37_3:142274740-142274749_Frame-Shift-Del-DEL-TTTTTTTTTT----': 4,
 'BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T': 16,
 'CASP5_GRCh37_11:104878041-104878050_Frame-Shift-Del-DEL-TTTTTTTTTT----': 3,
 'GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----': 7,
 'HMMR_GRCh37_5:162917426-162917434_In-Frame-Del-DEL-AAAAAAAAA----': 5,
 'KRAS_GRCh37_12:25398284-25398284_Missense-Mutation-SNP-C-T-T': 6,
 'MBD4_GRCh37_3:129155548-129155557_Frame-Shift-Del-DEL-TTTTTTTTTT----': 6,
 'MSH3_GRCh37_5:79970915-79970922_Frame-Shift-Del-DEL-AAAAAAAA----': 8,
 'MSH6_GRCh37_2:48030640-48030647_Frame-Shift-Del-DEL-CCCCCCCC----': 3,
 'PIK3CA_GRCh37_3:178952085-178952085_Missense-Mutation-SNP-A-G-G': 3,
 'PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----': 5,
 'RAD50_GRCh37_5:1319

In [2]:
#generate frequent 1-itemset L1
L1 = []
largestsupp = 0
for key in c1:
    if((c1[key]/numsamps) >= minconfidence): #if the 1-itemset is at least within confidence threshold, allow in
        L1.append([key])
    if((c1[key]/numsamps) > largestsupp):
        largestsupp = (c1[key]/numsamps)

print("Largest support of 1-itemset: ",largestsupp)
L1

Largest support of 1-itemset:  0.5


[['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T'],
 ['GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----'],
 ['MBD4_GRCh37_3:129155548-129155557_Frame-Shift-Del-DEL-TTTTTTTTTT----'],
 ['PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----'],
 ['HMMR_GRCh37_5:162917426-162917434_In-Frame-Del-DEL-AAAAAAAAA----'],
 ['SEC63_GRCh37_6:108214755-108214764_Frame-Shift-Del-DEL-TTTTTTTTTT----'],
 ['KRAS_GRCh37_12:25398284-25398284_Missense-Mutation-SNP-C-T-T'],
 ['RAD50_GRCh37_5:131931452-131931460_In-Frame-Del-DEL-AAAAAAAAA----'],
 ['ACVR2A_GRCh37_2:148683686-148683693_Frame-Shift-Del-DEL-AAAAAAAA----'],
 ['MSH3_GRCh37_5:79970915-79970922_Frame-Shift-Del-DEL-AAAAAAAA----'],
 ['ATR_GRCh37_3:142274740-142274749_Frame-Shift-Del-DEL-TTTTTTTTTT----'],
 ['TGFBR2_GRCh37_3:30691872-30691881_Frame-Shift-Del-DEL-AAAAAAAAAA----'],
 ['AIM2_GRCh37_1:159032487-159032496_Frame-Shift-Del-DEL-TTTTTTTTTT----'],
 ['TCF7L2_GRCh37_10:114925317-1

In [3]:

def can_prune(itemset, prev, k):
    subsets = set(itertools.combinations(itemset, k))
    for item in subsets: #parse through each subset of itemset
        s = []
        for l in item:
            s.append(l)
        s.sort()
        if s not in prev: #prev only contains items within threshhold
            return True
    return False

#generate k-itemset
def generate_set(prev, k):
    #given a list of initial (k-1) itemsets, fill result with viable (k) itemsets
    result = []
    for l1 in prev: #parse through itemsets n^2 and check support of pairs
        for l2 in prev:
            if(l1 == l2): #don't pair mutation with itself
                continue
            itemset = []
            count = 0
            while count < k-1:
                if l1[count] != l2[count]:
                    break
                else:
                    count += 1
            else:
                if l1[k-1] < l2[k-1]:
                    for item in l1:
                        itemset.append(item)
                    itemset.append(l2[k-1])
                    # get rid of subsets which don't meet threshhold
                    if not can_prune(itemset, prev, k):
                        result.append(itemset) 
                        itemset = [] 
    return result

#compute all frequent itemsets
def frequent_itemsets():
    k = 2
    Lk_1 = []
    Lk = []
    L = []
    count = 0
    transactions = 0
    for item in L1:
        Lk_1.append(item)
    while Lk_1 != []:
        Ck = []
        Lk = []
        Ck = generate_set(Lk_1, k-1)
        #print "-------------------------CANDIDATE %d-ITEMSET---------------------" % k
        #print "Ck: %s" % Ck
        #print "------------------------------------------------------------------"
        for c in Ck:
            count = 0
            s = set(c)
            for x in range(1,len(listit)):
                occurrances = []
                for y in range(1,len(listit[0])-1):
                    if(float(listit[x][y]) == 1.0):
                        occurrances.append(listit[0][y])
                t = set(occurrances)
                if s.issubset(t) == True:
                    count += 1
            if (count/numsamps) >= minsupport:
                c.sort()
                Lk.append(c)
        Lk_1 = []
        '''print("frequent %d-itemset:" % k)
        print(Lk)'''
        print("number of",k,"item-sets: ", len(Lk))
        for l in Lk:
            Lk_1.append(l)
        k += 1
        if Lk != []:
            L.append(Lk)
    
    return L


#calculate frequent item-sets
print("fequent item-sets:")
L = frequent_itemsets()
print()
print(L)



fequent item-sets:
number of 2 item-sets:  30
number of 3 item-sets:  19
number of 4 item-sets:  7
number of 5 item-sets:  1
number of 6 item-sets:  0

[[['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 'GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----'], ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 'PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----'], ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 'HMMR_GRCh37_5:162917426-162917434_In-Frame-Del-DEL-AAAAAAAAA----'], ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 'SEC63_GRCh37_6:108214755-108214764_Frame-Shift-Del-DEL-TTTTTTTTTT----'], ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T', 'RAD50_GRCh37_5:131931452-131931460_In-Frame-Del-DEL-AAAAAAAAA----'], ['BRAF_GRCh37_7:140453136

In [4]:
num = 0
counts = []
results = [["if this mutation occurs","then this mutation occurs","support","confidence"]]
for itemset in L:
    for item in itemset:
        count = 1
        while count < len(item):
            #get the subsets of the list to parse through
            subsets = set(itertools.combinations(item, count))
            count += 1
            for sub in subsets:
                occur1 = 0
                occur2 = 0
                s = []
                m = []
                for i in sub:
                    s.append(i)
                
                for x in range(1,len(listit)):
                    occurrances = []
                    for y in range(1,len(listit[0])-1):
                        if(float(listit[x][y]) == 1.0):
                            occurrances.append(listit[0][y])
                    if set(s).issubset(set(occurrances)) == True:
                        occur1 += 1
                    if set(item).issubset(set(occurrances)) == True:
                        occur2 += 1
                if (occur2/occur1) >= minconfidence:
                    for index in item:
                        if index not in s:
                            m.append(index)
                    print("Rule #%d : %s ==> %s support: %f confidence: %f\n" %(num, s, m, float(occur2/numsamps), float(occur2/occur1)))
                    crow = []
                    newrow = []
                    #write set 1
                    cell = ""
                    for i in range(0,len(s)):
                        if(i > 0):
                            cell += " and "
                        cell += s[i]
                    newrow.append(cell)
                    #write set 2
                    cell = ""
                    for i in range(0,len(m)):
                        if(i > 0):
                            cell += " and "
                        cell += m[i]
                    #get count of those with both anticedent and consequence
                    newrow.append(cell)
                    #write support and confidence
                    '''count = 0
                    for x in range(1,len(listit)):
                        hasall = True
                        for y in range(0,len(s)):
                            if(float(listit[x][listit[0].index(s[y])]) != 1.0):
                                hasall = False
                        for y in range(0,len(m)):
                            if(float(listit[x][listit[0].index(m[y])]) != 1.0):
                                hasall = False
                        if(hasall):
                            count += 1'''
                    newrow.append(str(float(occur2/numsamps)))
                    newrow.append(str(float(occur2/occur1)))
                    #newrow.append(str(count))
                    results.append(newrow)
                    num += 1 

Rule #0 : ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T'] ==> ['GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----'] support: 0.156250 confidence: 0.312500

Rule #1 : ['GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----'] ==> ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T'] support: 0.156250 confidence: 0.714286

Rule #2 : ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T'] ==> ['PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----'] support: 0.125000 confidence: 0.250000

Rule #3 : ['PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----'] ==> ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T'] support: 0.125000 confidence: 0.800000

Rule #4 : ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T'] ==> ['HMMR_GRCh37_5:16291742

In [5]:
results

[['if this mutation occurs',
  'then this mutation occurs',
  'support',
  'confidence'],
 ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
  'GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----',
  '0.15625',
  '0.3125'],
 ['GRB14_GRCh37_2:165365288-165365296_In-Frame-Del-DEL-TTTTTTTTT----',
  'BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
  '0.15625',
  '0.7142857142857143'],
 ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
  'PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----',
  '0.125',
  '0.25'],
 ['PRDM2_GRCh37_1:14108749-14108757_In-Frame-Del-DEL-AAAAAAAAA----',
  'BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
  '0.125',
  '0.8'],
 ['BRAF_GRCh37_7:140453136-140453136_Missense-Mutation-SNP-A-A-T_Missense-Mutation-SNP-A-T-T',
  'HMMR_GRCh37_5:162917426-162917434_In-Frame-De

In [6]:
with open("./rules.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(results)

'''#next make counts of each "item sets"
maxnum = 0
for x in range(1,len(results)):
    if(len(results[x][1].split(" and ")) > maxnum):
        maxnum = len(results[x][1].split(" and "))

icounts = []
alreadycounted = []
for k in range(0,maxnum):
    newrow = []
    newrow.append(str(k+1)+"-item sets")
    for j in range(1,len(results)):
        if(len(results[j][1].split(" and ")) == k+1):
            #correct num item-set, check if it's already been tallied
            if(set(results[j][1].split(" and ")) in alreadycounted):
                continue
            else:
                alreadycounted.append(set(results[j][1].split(" and ")))
            #still good, so calc up the number of occurrences
            count = 0
            for x in range(1,len(listit)):
                hasall = True
                for y in range(0,len(results[j][1].split(" and "))):
                    if(float(listit[x][listit[0].index(results[j][1].split(" and ")[y])]) != 1.0):
                        hasall = False
                if(hasall):
                    count += 1
            #now we have the count, make the string to output
            mystring = "("
            for y in range(0,len(results[j][1].split(" and "))):
                mystring += results[j][1].split(" and ")[y].split("_")[0]
                if(y != len(results[j][1].split(" and "))-1):
                    mystring += " and "
            mystring += "): count: "
            mystring += str(count)
            newrow.append(mystring)
    icounts.append(newrow)
    
    
with open("./itemcounts.csv","w") as fun:
    writer = csv.writer(fun)
    writer.writerows(icounts)

icounts'''

'#next make counts of each "item sets"\nmaxnum = 0\nfor x in range(1,len(results)):\n    if(len(results[x][1].split(" and ")) > maxnum):\n        maxnum = len(results[x][1].split(" and "))\n\nicounts = []\nalreadycounted = []\nfor k in range(0,maxnum):\n    newrow = []\n    newrow.append(str(k+1)+"-item sets")\n    for j in range(1,len(results)):\n        if(len(results[j][1].split(" and ")) == k+1):\n            #correct num item-set, check if it\'s already been tallied\n            if(set(results[j][1].split(" and ")) in alreadycounted):\n                continue\n            else:\n                alreadycounted.append(set(results[j][1].split(" and ")))\n            #still good, so calc up the number of occurrences\n            count = 0\n            for x in range(1,len(listit)):\n                hasall = True\n                for y in range(0,len(results[j][1].split(" and "))):\n                    if(float(listit[x][listit[0].index(results[j][1].split(" and ")[y])]) != 1.0):\n   